In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import numpy as np
import sys
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
sys.path.insert(0,'/content/drive/MyDrive/mini_project_full')
from utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import h5py

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

import tensorflow.compat.v1 as tf

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [3]:
def LeNet(input_shape = (64, 64, 1), classes = 6):
    
    X_input = Input(input_shape)
    
    X = Conv2D(6, (3,3), strides = (1,1), name = 'conv1', kernel_initializer = glorot_uniform())(X_input)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = AveragePooling2D((3, 3))(X)
    
    X = Conv2D(16, (3,3), strides = (1,1), name = 'conv2', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv2')(X)
    X = Activation('relu')(X)
    X = AveragePooling2D((3, 3))(X)
    
    X = Flatten()(X)
    X = Dense(120, activation='relu', name='fc2', kernel_initializer = glorot_uniform())(X)
    X = Dense(84, activation='relu', name='fc3', kernel_initializer = glorot_uniform())(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform())(X)
    
    model = Model(inputs = X_input, outputs = X, name='LeNet')

    return model

In [4]:
LeNet = LeNet(input_shape = (280, 320, 1), classes = 6)

In [5]:
LeNet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig/255.
X_test = X_test_orig/255.

Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

X_train1 = np.zeros((X_train.shape[0], 280, 320))
X_test1 = np.ones((X_test.shape[0], 280, 320))
rgb_weights = [0.2989, 0.5870, 0.1140]
  
X_train = np.dot(X_train[...,:3], rgb_weights)
for i in range(X_train.shape[0]):
    X_train1[i] = cv2.resize(X_train[i], (320,280))
X_test = np.dot(X_test[...,:3], rgb_weights)
for i in range(X_test.shape[0]):
    X_test1[i] = cv2.resize(X_test[i], (320,280))


print ("number of training examples = " + str(X_train1.shape[0]))
print ("number of test examples = " + str(X_test1.shape[0]))
print ("X_train shape: " + str(X_train1.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test1.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 280, 320)
Y_train shape: (1080, 6)
X_test shape: (120, 280, 320)
Y_test shape: (120, 6)


In [7]:
LeNet.fit(X_train1, Y_train, epochs = 25, batch_size = 32)

Epoch 1/25
34/34 [==============================] - 14s 32ms/step - loss: 1.3245 - accuracy: 0.5417
Epoch 2/25
34/34 [==============================] - 1s 25ms/step - loss: 0.4481 - accuracy: 0.8269
Epoch 3/25
34/34 [==============================] - 1s 25ms/step - loss: 0.2138 - accuracy: 0.9269
Epoch 4/25
34/34 [==============================] - 1s 25ms/step - loss: 0.1908 - accuracy: 0.9370
Epoch 5/25
34/34 [==============================] - 1s 25ms/step - loss: 0.0947 - accuracy: 0.9759
Epoch 6/25
34/34 [==============================] - 1s 24ms/step - loss: 0.0654 - accuracy: 0.9824
Epoch 7/25
34/34 [==============================] - 1s 25ms/step - loss: 0.0255 - accuracy: 0.9981
Epoch 8/25
34/34 [==============================] - 1s 25ms/step - loss: 0.0211 - accuracy: 0.9963
Epoch 9/25
34/34 [==============================] - 1s 25ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 10/25
34/34 [==============================] - 1s 24ms/step - loss: 0.0073 - accuracy: 1.0000
Epoch 11

In [8]:
print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Number of GPUs Available:  1


In [9]:
preds = LeNet.evaluate(X_test1, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 0s 14ms/step - loss: 0.2956 - accuracy: 0.9250
Loss = 0.295556902885437
Test Accuracy = 0.925000011920929


In [10]:
LeNet.save("/content/drive/MyDrive/mini_project_full/LeNet_model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/mini_project_full/LeNet_model/assets
